<a href="https://colab.research.google.com/github/quantranvr/all-in-one/blob/main/QA_with_RAG_series_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

Tutorial @ https://python.langchain.com/docs/use_cases/question_answering/sources

This notebook contains 2 core parts:
1. **Reproduce** [tutorial](https://python.langchain.com/docs/use_cases/question_answering/sources)'s example
2. **Apply** knowledge learned to solve a similar problem

# Installation

In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai chromadb bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Part 1: Reproduce

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# load document
web_paths = (
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
)

bs_strainer = bs4.SoupStrainer(
    class_ = ("post-content", "post-title", "post-header")
)

loader = WebBaseLoader(
    web_paths = web_paths,
    bs_kwargs = {"parse_only": bs_strainer},
)

docs = loader.load()

In [ ]:
# split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

splits = text_splitter.split_documents(docs)

In [ ]:
# store in vectorstore
vectorstore = Chroma.from_documents(
    documents = splits,
    embedding = OpenAIEmbeddings(),
)

In [ ]:
# retrieve and generate
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# chain returns answer without sources
chain_wo_sources = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain_wo_sources.invoke("What is Task Decomposition?")

"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It can be done through various methods such as using prompting techniques, task-specific instructions, or human inputs. The goal is to make the task more manageable and facilitate the interpretation of the model's thinking process."

In [ ]:
# chain returns answer with sources
chain_w_sources = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(
    answer = (
        RunnablePassthrough.assign(context=lambda x: format_docs(x["context"]))
        | prompt
        | llm
        | StrOutputParser()
    )
)

In [ ]:
chain_w_sources.invoke("What is Task Decomposition?")

{'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}),
  Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree st

# Part 2: Apply

Problem:

A LangChain learner want to solidify his knowledge on LangChain's Agents.

Build a chatbot that is able to answer his questions with reliable sources

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.prompts import PromptTemplate

In [ ]:
web_paths = (
    "https://python.langchain.com/docs/modules/agents/",
    "https://python.langchain.com/docs/modules/agents/quick_start",
    "https://python.langchain.com/docs/modules/agents/concepts",

    "https://python.langchain.com/docs/modules/agents/agent_types/",
    "https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent",
    "https://python.langchain.com/docs/modules/agents/agent_types/openai_tools",
    "https://python.langchain.com/docs/modules/agents/agent_types/xml_agent",
    "https://python.langchain.com/docs/modules/agents/agent_types/json_agent",
    "https://python.langchain.com/docs/modules/agents/agent_types/structured_chat",
    "https://python.langchain.com/docs/modules/agents/agent_types/react",
    "https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search",

    "https://python.langchain.com/docs/modules/agents/how_to/custom_agent",
    "https://python.langchain.com/docs/modules/agents/how_to/streaming",
    "https://python.langchain.com/docs/modules/agents/how_to/agent_iter",
    "https://python.langchain.com/docs/modules/agents/how_to/agent_structured",
    "https://python.langchain.com/docs/modules/agents/how_to/handle_parsing_errors",
    "https://python.langchain.com/docs/modules/agents/how_to/intermediate_steps",
    "https://python.langchain.com/docs/modules/agents/how_to/max_iterations",
    "https://python.langchain.com/docs/modules/agents/how_to/max_time_limit",
    "https://python.langchain.com/docs/modules/agents/how_to/streaming_events",

    "https://python.langchain.com/docs/modules/agents/tools/",
    "https://python.langchain.com/docs/modules/agents/tools/toolkits",
    "https://python.langchain.com/docs/modules/agents/tools/custom_tools",
    "https://python.langchain.com/docs/modules/agents/tools/tools_as_openai_functions",
)

bs4_strainer = bs4.SoupStrainer(class_=("theme-doc-markdown markdown"))

loader = WebBaseLoader(
    web_paths = web_paths,
    bs_kwargs = {"parse_only": bs4_strainer},
)

docs = loader.load()

In [ ]:
print(f"Number of loaded documents = {len(docs)}")

Number of loaded documents = 24


In [ ]:
def format_loaded_doc(doc):
    return doc.replace("\n", " ")

format_loaded_doc(docs[0].page_content)

for doc in docs:
    doc.page_content = format_loaded_doc(doc.page_content)

# print("Example of loaded docs:")
# print(f"({len(docs[1].page_content)} characters)")
# docs[0]

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True
)

splits = splitter.split_documents(docs)

In [ ]:
vectorstore = Chroma.from_documents(
    documents = splits,
    embedding = OpenAIEmbeddings(),
)

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

answer_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    # | StrOutputParser()
)

rag_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=answer_chain)

In [ ]:
questions = [
    "How to create my own custom agent?",
    "What is Agent Finish?",
    "What are the differences between agents and chains?",
    "Is it possible that an agent could return structured output?",
    "Is it possible that an agent runs into infinite loop?"
]

# answer the original question
llm_output = rag_chain.invoke(questions[-1])

In [ ]:
context = llm_output["context"]
question = llm_output["question"]
answer = llm_output["answer"]
print(answer.content)

Yes, it is possible for an agent to run into an infinite loop. In the given context, an adversarial example is used to trick the agent into continuing forever.


In [ ]:
# Filter correct references
prompt_template = PromptTemplate.from_template(
    """
    Could this statement: {statement}
    be deduced from this text: {text}
    Answer either YES or NO!
    """
)

refs = []
for ref in context:
    prompt = prompt_template.format(
        statement = answer.content,
        text = ref.page_content
    )

    response = llm.invoke(prompt)

    if "yes" in response.content.lower():
        refs.append(ref)

In [ ]:
# Extract sentences from the references that support the answer
prompt_template = PromptTemplate.from_template(
    """
    The following statement:
    {statement}
    is deduced from the following text:
    {text}
    The proof of the statement can be found in which sentences in the text?
    """
)

for ref in refs:
    prompt = prompt_template.format(
        statement = answer.content,
        text = ref.page_content
    )

    response = llm.invoke(prompt)

    print(f"Prompt:{prompt}")
    print(f"Answer:\n{response.content}")
    print(f"\nLink: {ref.metadata['source']}")

Prompt:
    The following statement:
    Yes, it is possible for an agent to run into an infinite loop. In the given context, an adversarial example is used to trick the agent into continuing forever.
    is deduced from the following text:
    Cap the max number of iterationsThis notebook walks through how to cap an agent at taking a certain number of steps. This can be useful to ensure that they do not go haywire and take too many steps.from langchain import hubfrom langchain.agents import AgentExecutor, create_react_agentfrom langchain_community.tools import WikipediaQueryRunfrom langchain_community.utilities import WikipediaAPIWrapperfrom langchain_openai import ChatOpenAIapi_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)tool = WikipediaQueryRun(api_wrapper=api_wrapper)tools = [tool]# Get the prompt to use - you can modify this!prompt = hub.pull("hwchase17/react")llm = ChatOpenAI(temperature=0)agent = create_react_agent(llm, tools, prompt)First, let‚Äôs d